# Programação Genética - Trabalho Final
## Leonardo Augusto Ferreira
### leauferreira@cpdee.ufmg.br

## Alunos:
### Gabriel Camatta Zanotelli - 2018020140
### Lucas de Almeida Martins - 2018020328

In [ ]:
# Anaconda
# !conda install gplearn
# !conda install eckity
# !conda install pmlb

# python - jupyter nootebook - colab
# !pip install gplearn
# !pip install eckity
# !pip install pmlb


# documentation:
# https://docs.sympy.org/latest/install.html
# https://github.com/EC-KitY/EC-KitY
# https://epistasislab.github.io/pmlb/

In [ ]:
from pmlb import fetch_data
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from eckity.algorithms.simple_evolution import SimpleEvolution
from eckity.sklearn_compatible.sk_classifier import SKClassifier
from eckity.breeders.simple_breeder import SimpleBreeder
from eckity.creators.gp_creators.ramped_hh import RampedHalfAndHalfCreator
from eckity.genetic_encodings.gp.tree.functions import f_add, f_mul, f_sub, f_div, f_neg, f_sqrt, f_log, f_abs, f_inv, f_max, \
    f_min
from eckity.genetic_encodings.gp.tree.utils import create_terminal_set
from eckity.genetic_operators.crossovers.subtree_crossover import SubtreeCrossover
from eckity.genetic_operators.mutations.subtree_mutation import SubtreeMutation
from eckity.genetic_operators.selections.tournament_selection import TournamentSelection
from eckity.statistics.best_avg_worst_size_tree_statistics import BestAverageWorstSizeTreeStatistics
from eckity.subpopulation import Subpopulation
from eckity.termination_checkers.threshold_from_target_termination_checker import ThresholdFromTargetTerminationChecker

# Adding your own functions
from eckity.sklearn_compatible.classification_evaluator import ClassificationEvaluator


In [ ]:
x, y = fetch_data('breast', return_X_y=True)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=.7)

In [ ]:
terminal_set = create_terminal_set(x_train)
# function_set = [f_add, f_mul, f_sub, f_div, f_sqrt, f_log, f_abs, f_neg, f_inv, f_max, f_min]

function_set = [f_add, f_mul, f_sub, f_div]

algo = SimpleEvolution(
    Subpopulation(
        creators=RampedHalfAndHalfCreator(init_depth=(2, 4),
                                                    terminal_set=terminal_set,
                                                    function_set=function_set,
                                                    bloat_weight=0.0001),
        population_size=1000,
        evaluator=ClassificationEvaluator(),
        higher_is_better=True,
        elitism_rate=0.05,
        operators_sequence=[
            SubtreeCrossover(probability=0.9, arity=2),
            SubtreeMutation(probability=0.2, arity=1)
        ],
        selection_methods=[
            # (selection method, selection probability) tuple
            (TournamentSelection(tournament_size=4, higher_is_better=True), 1)
        ]
    ),
    breeder=SimpleBreeder(),
    max_workers=1,
    max_generation=100,

    termination_checker=ThresholdFromTargetTerminationChecker(optimal=1, threshold=0.03),
    statistics=BestAverageWorstSizeTreeStatistics()
)

classifier = SKClassifier(algo)

classifier.fit(x_train, y_train)

In [ ]:
acc = accuracy_score(y_test, classifier.predict(x_test))
print(acc)

## Comparação de Desempenho entre as Bibliotecas de Programação Genética EC-KitY e GPlearn utilizando Datasets de Regressão e Classificação

Para o trabalho final, você deverá utilizar as bibliotecas de programação Genética EC-KitY e GPlearn, juntamente com a biblioteca Penn Machine Learning Benchmarks, a fim de gerar um código Python que realizará testes em três conjuntos de dados para regressão e três conjuntos de dados para classificação. Seu objetivo é comparar as métricas obtidas e determinar qual biblioteca apresenta os melhores resultados.

Durante o estudo, você deverá variar as probabilidades de mutação e cruzamento, a fim de identificar a melhor configuração para cada uma das bibliotecas. Registre cuidadosamente os resultados obtidos em relação às métricas avaliadas, considerando medidas como acurácia, precisão, recall, F1-score, erro médio quadrático (RMSE) ou outras relevantes para o problema em questão.

Ao final do estudo, apresente uma análise comparativa dos resultados para cada biblioteca, destacando as probabilidades de mutação e cruzamento que produziram os melhores desempenhos. Utilize gráficos, tabelas ou outras visualizações que considerar pertinentes para ilustrar seus resultados e facilitar a compreensão.

---
# Relatório

Nesta atividade analisamos a eficiência das bibliotecas de Programação Genética *EC-KitY* e *GPlearn* de forma a fazer uma análise quantitativa da eficiência de ambas, por meio da utilização de diversos conjuntos de dados disponíveis na biblioteca *pmlb*.

Esta análise será feita utilizando ambas as bibliotecas para resolução de problemas binários e mono objetivo de quatro conjuntos de dados selecionados, variando dois parâmetros de imensa importância: a probabilidade de mutação e cruzamento. Para cada combinação, serão realizadas diversas iterações, de forma a obter um resultado médio.


### Dependencias

In [13]:
from pmlb import fetch_data
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, mean_squared_log_error
from gplearn.genetic import SymbolicRegressor
from eckity.algorithms.simple_evolution import SimpleEvolution
from eckity.sklearn_compatible.sk_classifier import SKClassifier
from eckity.breeders.simple_breeder import SimpleBreeder
from eckity.creators.gp_creators.ramped_hh import RampedHalfAndHalfCreator
from eckity.genetic_encodings.gp.tree.functions import f_add, f_mul, f_sub, f_div, f_min, f_max, f_sin, f_cos, f_log
from eckity.genetic_encodings.gp.tree.utils import create_terminal_set
from eckity.genetic_operators.crossovers.subtree_crossover import SubtreeCrossover
from eckity.genetic_operators.mutations.subtree_mutation import SubtreeMutation
from eckity.genetic_operators.selections.tournament_selection import TournamentSelection
from eckity.statistics.best_avg_worst_size_tree_statistics import BestAverageWorstSizeTreeStatistics
from eckity.subpopulation import Subpopulation
from eckity.termination_checkers.threshold_from_target_termination_checker import ThresholdFromTargetTerminationChecker
from eckity.sklearn_compatible.classification_evaluator import ClassificationEvaluator

### Conjuntos de dados

In [14]:
x1, y1 = fetch_data('diabetes', return_X_y=True)
x2, y2 = fetch_data('cloud', return_X_y=True)
x3, y3 = fetch_data('iris', return_X_y=True)

### Desenvolvimento

Para ambas as bibliotecas serão adotados alguns parâmetros padrõesao longo dos testes realizados.
- **EC-KitY**
    - População máxima de 1000 indivíduos
    - Mesmo conjunto de equações para a resolução do problema (adição, subtração, multiplicação, divisão, máximo, mínimo, seno, cosseno, logarítmo)
    - Valores base de probabilidade de mutação como 70% e probabilidade de cruzamento e 100%
    - Serão testados valores da probabilidade de mutação de 30-90%, com um passo de 20%
    - Serão testados valores da probabilidade de cruzamento de 40-100%, com um passo de 20%
    - Máximo de 100 gerações
    - Para cada caso, serão feitas um número de execuções de garantam uma certa segurança dos resultados. Para essa ativiade, serão feitas 10 iterações

- **GPLearn**
    - População máxima de 1000 indivíduos
    - Mesmo conjunto de equações para a resolução do problema (adição, subtração, multiplicação, divisão, máximo, mínimo, seno, cosseno, logarítmo)
    - Como a biblioteca exige que a somatória das probabilidade de corssover e mutações (*Subtree*, *hois* e *point*) tenham uma somatória total de 1, o programa será executado 4 vezes, sempre com a probabilidade iguais dos três tipos de mutação. As configurações são:
        - Probabilidade de corssover de 90%; probabilidade de mutação de 3%
        - Probabilidade de corssover de 80%; probabilidade de mutação de 6,67%
        - Probabilidade de corssover de 70%; probabilidade de mutação de 10%
        - Probabilidade de corssover de 60%; probabilidade de mutação de 13,33%
    - Máximo de 100 gerações
    - Para cada caso, serão feitas um número de execuções de garantam uma certa segurança dos resultados. Para essa ativiade, serão feitas 10 iterações

### Set-up

In [15]:
function_set_ec_kity = [f_add, f_mul, f_sub, f_div, f_min, f_max, f_sin, f_cos, f_log]
function_set_gp_learn = ['add', 'sub', 'mul', 'div', 'min', 'max', 'sin', 'cos', 'log']
mutation_vec  = [0.3, 0.5, 0.7, 0.9]
crossover_vec = [0.4, 0.6, 0.8, 1]
iterations = 10

### EC-KitY

In [16]:
def ec_kity(x, y, _mutation = 0.7, _crossover =1, _higher_is_better = True, _populationSize = 1000, _generationsMax = 100, _average="binary"):

    x_train, x_test, y_train, y_test = train_test_split(x, y,)
    terminal_set = create_terminal_set(x_train)

    algo = SimpleEvolution(
        Subpopulation(
            creators=RampedHalfAndHalfCreator(init_depth=(2, 4),
                                                        terminal_set=terminal_set,
                                                        function_set=function_set_ec_kity,
                                                        bloat_weight=0.0001),
            population_size=_populationSize,
            evaluator=ClassificationEvaluator(),
            higher_is_better=_higher_is_better,
            elitism_rate=0.05,
            operators_sequence=[
                SubtreeCrossover(probability=_mutation, arity=2),
                SubtreeMutation(probability=_crossover, arity=1)
            ],
            selection_methods=[
                # (selection method, selection probability) tuple
                (TournamentSelection(tournament_size=4, higher_is_better=True), 1)
            ]
        ),
        breeder=SimpleBreeder(),
        max_workers=1,
        max_generation=_generationsMax,

        termination_checker=ThresholdFromTargetTerminationChecker(optimal=1, threshold=0.03),
        statistics=BestAverageWorstSizeTreeStatistics()
    )

    classifier = SKClassifier(algo)

    classifier.fit(x_train, y_train)

    acc =accuracy_score(y_test, classifier.predict(x_test))
    f1 = f1_score(y_test, classifier.predict(x_test), average=_average)
    rec = recall_score(y_test, classifier.predict(x_test), average=_average)
    msle = mean_squared_log_error(y_test, classifier.predict(x_test))

    print("Acuracia: ", acc)
    print("F1: ", f1)
    print("Recall: ", rec)
    print("RMSE: ", msle)

    return [acc, f1, rec, msle]

### GPLearn

In [17]:
def gp_learn(x, y, _mutation = 0.1, _crossover = 0.7, _higher_is_better = True, _populationSize = 1000, _generationsMax = 100, _average="binary"):

    x_train, x_test, y_train, y_test = train_test_split(x, y)

    gplearn_model = SymbolicRegressor(population_size=_populationSize,
                                      generations=_generationsMax,
                                      function_set=function_set_gp_learn,
                                      stopping_criteria=0.01,
                                      p_crossover=_crossover,
                                      p_subtree_mutation=_mutation,
                                      p_hoist_mutation=_mutation,
                                      p_point_mutation=_mutation,
                                      verbose=1,
                                      parsimony_coefficient=0.01)

    gplearn_model.fit(x_train, y_train)

    acc =accuracy_score(y_test, gplearn_model.predict(x_test))
    f1 = f1_score(y_test, gplearn_model.predict(x_test), average=_average)
    rec = recall_score(y_test, gplearn_model.predict(x_test), average=_average)
    msle = mean_squared_log_error(y_test, gplearn_model.predict(x_test))

    print("Acuracia: ", acc)
    print("F1: ", f1)
    print("Recall: ", rec)
    print("RMSE: ", msle)

    return [acc, f1, rec, msle]

# Execução

In [ ]:
diabetes_ec_kity_cross_results = []; diabetes_ec_kity_mutt_results  = []
diabetes_gp_learn_cross_results = []; diabetes_gp_learn_mutt_results  = []

for cross in crossover_vec:
    diabetes_ec_kity_cross_results.append([ec_kity(x1, y1, _crossover=cross, _higher_is_better=True) for _ in range(iterations)])
for mut in mutation_vec:
    diabetes_ec_kity_mutt_results.append( [ec_kity(x1, y1, _mutation=mut, _higher_is_better=True) for _ in range(iterations)])

In [ ]:
cloud_ec_kity_cross_results = []; cloud_ec_kity_mutt_results  = []
cloud_gp_learn_cross_results = []; cloud_gp_learn_mutt_results  = []

for cross in crossover_vec:
    cloud_ec_kity_cross_results.append([ec_kity(x2, y2, _crossover=cross, _higher_is_better=False) for _ in range(iterations)])
for mut in mutation_vec:
    cloud_ec_kity_mutt_results.append( [ec_kity(x2, y2, _mutation=mut, _higher_is_better=False) for _ in range(iterations)])

In [ ]:
iris_ec_kity_cross_results = []; iris_ec_kity_mutt_results  = []
iris_gp_learn_cross_results = []; iris_gp_learn_mutt_results  = []

for cross in crossover_vec:
    iris_ec_kity_cross_results.append([ec_kity(x3, y3, _crossover=cross, _higher_is_better=True) for _ in range(iterations)])
for mut in mutation_vec:
    iris_ec_kity_mutt_results.append( [ec_kity(x3, y3, _mutation=mut, _higher_is_better=True) for _ in range(iterations)])

# Resultados das Execuções


## EC Kity
---
### Variação do Crossover
| Métrica       |  Prob. de Crossover  | Diabetes              | Cloud               | Iris                 |
|:--------------|:--------------------:|-----------------------|---------------------|----------------------|
| **Acuracia**  |         40%          | 0.6875                | 0.2222222222222222  | 0.631578947368421    |
|               |         60%          | 0.6354166666666666    | 0.2222222222222222  | 0.631578947368421    |
|               |         80%          | 0.6614583333333334    | 0.3333333333333333  | 0.7105263157894737   |
|               |         100%         | 0.6666666666666666    | 0.37037037037037035 | 0.631578947368421    |
| **F1**        |         40%          | 0.8148148148148148    | 0.11991711991711991 | 0.511842105263158    |
|               |         60%          | 0.7770700636942676    | 0.15959595959595962 | 0.5279605263157895   |
|               |         80%          | 0.7962382445141065    | 0.24603174603174605 | 0.6028151774785802   |
|               |         100%         | 0.8                   | 0.22866344605475042 | 0.5152354570637119   |
| **Recall**    |         40%          | 1.0                   | 0.2222222222222222  | 0.631578947368421    |
|               |         60%          | 1.0                   | 0.2222222222222222  | 0.631578947368421    |
|               |         80%          | 1.0                   | 0.3333333333333333  | 0.7105263157894737   |
|               |         100%         | 1.0                   | 0.37037037037037035 | 0.631578947368421    |
| **RMSE**      |         40%          | 0.05137561059161424   | 0.888038084024321   | 0.06056914090800835  |
|               |         60%          | 0.059938212356883264  | 0.6032837055643817  | 0.06056914090800835  |
|               |         80%          | 0.05565691147424875   | 0.4605378919476176  | 0.04759003928486371  |
|               |         100%         | 0.054800651297721846  | 0.7078889947678819  | 0.06056914090800835  |


---
### Variação da Mutação
| Métrica       | Prob. de Mutação  | Diabetes             | Cloud               | Iris                 |
|:--------------|:-----------------:|----------------------|---------------------|----------------------|
| **Acuracia**  |        30%        | 0.6666666666666666   | 0.4444444444444444  | 0.6578947368421053   |
|               |        50%        | 0.7135416666666666   | 0.18518518518518517 | 0.6578947368421053   |
|               |        70%        | 0.6510416666666666   | 0.4074074074074074  | 0.6842105263157895   |
|               |        90%        | 0.6302083333333334   | 0.37037037037037035 | 0.5789473684210527   |
| **F1**        |        30%        | 0.8                  | 0.3440962329851218  | 0.5503759398496241   |
|               |        50%        | 0.8328267477203648   | 0.13001101788287142 | 0.5585738539898133   |
|               |        70%        | 0.7886435331230284   | 0.31465653687875905 | 0.5789473684210527   |
|               |        90%        | 0.7731629392971245   | 0.26492374727668844 | 0.45706371191135736  |
| **Recall**    |        30%        | 1.0                  | 0.4444444444444444  | 0.6578947368421053   |
|               |        50%        | 1.0                  | 0.18518518518518517 | 0.6578947368421053   |
|               |        70%        | 1.0                  | 0.4074074074074074  | 0.6842105263157895   |
|               |        90%        | 1.0                  | 0.37037037037037035 | 0.5789473684210527   |
| **RMSE**      |        30%        | 0.054800651297721846 | 0.5346983748746058  | 0.05624277370029348  |
|               |        50%        | 0.04709430970897971  | 0.6506198938454402  | 0.05624277370029348  |
|               |        70%        | 0.05736943182730255  | 0.38979051403507503 | 0.05191640649257859  |
|               |        90%        | 0.06079447253341017  | 0.6506198938454402  | 0.06922187532343811  |


## GP-Learn
---
### Variação do Crossover
| Métrica       | Prob. Cossover - Mutação | Diabetes | Cloud | Iris |
|:--------------|:------------------------:|----------|-------|------|
| **Acuracia**  |         90% - 3%         |          |       |      |
|               |       80% - 6,67%        |          |       |      |
|               |        70% - 10%         |          |       |      |
|               |       60% - 13,33%       |          |       |      |
| **F1**        |         90% - 3%         |          |       |      |
|               |       80% - 6,67%        |          |       |      |
|               |        70% - 10%         |          |       |      |
|               |       60% - 13,33%       |          |       |      |
| **Recall**    |         90% - 3%         |          |       |      |
|               |       80% - 6,67%        |          |       |      |
|               |        70% - 10%         |          |       |      |
|               |       60% - 13,33%       |          |       |      |
| **RMSE**      |         90% - 3%         |          |       |      |
|               |       80% - 6,67%        |          |       |      |
|               |        70% - 10%         |          |       |      |
|               |       60% - 13,33%       |          |       |      |



In [ ]:
gp_learn(x3, y3, _crossover=.1/3, _mutation=.9, _higher_is_better=True, _average='weighted')

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    17.23          19.8833        6         0.238885              N/A      1.50m
   1     9.14          2.45294       11         0.194071              N/A      1.27m
   2     8.37          2.36943        9         0.190486              N/A      1.10m
   3     6.77           2.9631        9         0.185422              N/A      1.09m
   4     6.23          2.82623        9         0.185422              N/A      1.08m
   5     6.29          2.95463       13         0.182484              N/A      1.05m
   6     6.26          3.92686       19         0.185422              N/A      1.04m
   7     6.00          3.24538       13         0.181883              N/A      1.01m
   8     6.22          3.77107        3         0.185061              N/A  

### Anotações

Bases de dados de classificação tem que ser binaria
As de regressão tem que ser mono objetivo

Usar a biblioteca "pmlb" pra pegar um banco de dados

Variar as probabilidades ditas no enunciado

Relatorio no markdown

Rodar umas 30 (ou 3) vezes pra cada resultado (uma quantidade que de uma certa segurnaça) uma vez que o resultado é probabilistico

Pegar uns 3 ou 4 valores diferentes para cada